<a href="https://colab.research.google.com/github/davidwhogg/GenerativeVsDiscriminative/blob/master/ipynb/generative_vs_discriminative.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative *vs* Discriminative models for inference

- Do generative models generally outperform discriminative models in various ways?
- How do both of these compare to information-theory-optimal estimators?
- Are generative models less subject to adversarial attack?
- Are generative and discriminative models equally good for de-noising noisy labels?
- ... and related matters.

## authors
- **David W Hogg** *(NYU) (MPIA) (Flatiron)*
- **Soledad Villar** *(NYU)*

## license
Copyright 2019, 2020 the authors. All rights reserved (for now).


In [0]:
import numpy as np
import pylab as plt
#matplotlib inline

In [0]:
# set up the data-generating matrices of God
# BUG: This should probably be a function call.

# set God's integers
maximalD = 2 ** 10 # ESLII calls this "p"
fiducialD = 2 ** 8
M = 1 # ESLII always sets this to 1
K = 42
assert(M <= K)

# set training set size
maximalN = maximalD
fiducialN = 2 ** 8
Ntest = 2 ** 12 # number of points to use to compute biases and variances
Ntrial = 2 ** 4 # number of training trials to use to compute biases and variances

# set God's matrices
# the true world can be a (K-1)-order Legendre (orthogonal) or Chebyshev (non-orthogonal) polynomial
#Q = np.vstack([np.polynomial.legendre.legval(np.arange(-1. + 1./D, 1., 2./D),
#                                             np.eye(K)[k])
#    for k in range(K)]).T
# or the true world can be a set of random matrices
np.random.seed(42)
maximalQ = np.random.normal(size=(maximalD, K))
np.random.seed(17)
P = np.random.normal(size=(M, K))

# We aren't being very general about the noise
maximalxivars = np.zeros(maximalD) + 2. ** 4 # inverse of diagonal of C_x ; all C_xn identical and proportional to I
yivars = np.zeros(M) + 2. ** 4 # inverse of diagonal of C_y; same for C_yn

In [0]:
# define functions to take and use pseudo-inverses.

def get_lambda(M, maxcond, lam):
  """
  internal function for pseudo_inverse() and pseudo_solve()
  """
  if lam is not None:
    assert maxcond is None, "get_lambda(): You can't set both maxcond and lam"
    return lam
  if maxcond is None:
    return 0.
  assert maxcond > 1., "get_lambda(): maxcond must be > 1"
  return np.max(np.linalg.eigvalsh(M)) / maxcond

def pseudo_solve(A, x, Cinv=None, weights=None, maxcond=None, lam=None):
  """
  return solve(ATA, ATx) but be somewhat clever about numerics

  ## inputs:
  - A: rectangular matrix
  - Cinv (optional): square non-negative semi-definite matrix that can multiply
      AT on the right or A on the left.
  - weight (optional): diagonal of Cinv if Cinv is diagonal (conflicts with
      Cinv).
  - maxcond (optional): a regularization parameter >> 1; costs a eigvalsh()
      call.
  - lam (optional): a regularization parameter (conflicts with maxcond).

  ## outputs:
  - Pseudo-inverse of A applied to x.
  """
  foo, bar = A.shape # need this to decide whether to do ATA or AAT

  # use weights or Cinv to weight A.T
  if weights is not None:
    assert Cinv is None, "pseudo_inverse(): You can't set both Cinv and weight"
    wAT = weights * A.T
  elif Cinv is not None:
    wAT = A.T @ Cinv
  else:
    wAT = A.T

  if foo < bar:
    AwAT = A @ wAT
    lamb = get_lambda(AwAT, maxcond, lam)
    if x is None:
      return np.linalg.solve(AwAT + lamb * np.eye(foo), wAT.T).T
    return wAT @ np.linalg.solve(AwAT + lamb * np.eye(foo), x) # There are many ways to write this line

  if x is None:
    wATx = wAT
  else:
    wATx = wAT @ x

  wATA = wAT @ A
  lamb = get_lambda(wATA, maxcond, lam)
  return np.linalg.solve(wATA + lamb * np.eye(bar), wATx)

def pseudo_inverse(A, Cinv=None, weights=None, maxcond=None, lam=None):
  """
  return solve(ATA, AT) but be somewhat clever about numerics
  """
  return pseudo_solve(A, None, Cinv=Cinv, weights=weights, maxcond=maxcond, lam=lam)

In [0]:
def gods_estimator(P, Q, xivars):
  """
  make the best possible estimator
  (this, we hope, is what everything approaches, in the limit)
  note super-safe linear algebra stupidity
  """
  return P @ pseudo_inverse(Q, weights=xivars, maxcond=1e9)

In [0]:
maximalW = gods_estimator(P, maximalQ, maximalxivars)

In [0]:
def make_data_set(N, P, Q, xivars, yivars):
  """
  actually make the random data, using the matrices of God
  """
  M, K = P.shape
  D, KK = Q.shape
  assert K == KK
  zs = np.random.normal(size=(N, K))
  godxs = (Q @ zs.T).T # "true" xs
  xs = godxs + np.random.normal(size=(N, D)) / np.sqrt(xivars)
  godys = (P @ zs.T).T # "true" ys
  ys = godys + np.random.normal(size=(N, M)) / np.sqrt(yivars)
  return xs, ys

In [0]:
# make maximal train and test sets
# NOTE THAT THE TEST Y HAS INFINITE yivars OR ZERO NOISE
np.random.seed(8675309 + 1)
maximalX = np.zeros((Ntrial, maximalN, maximalD))
maximalY = np.zeros((Ntrial, maximalN, M))
for trial in range(Ntrial):
  maximalX[trial], maximalY[trial] = make_data_set(maximalN, P, maximalQ, maximalxivars, yivars)
maximalXtest, maximalYtest = make_data_set(Ntest, P, maximalQ, maximalxivars, np.inf * yivars)
print(maximalX.shape, maximalY.shape)

In [0]:
#u, s, v = np.linalg.svd(maximalX, full_matrices=False)
#plt.plot(s, "ko")
#plt.semilogy()

In [0]:
#examples = np.arange(4)
#colors=['red', 'purple', 'green', 'blue']
#for i,n in enumerate(examples):
#  plt.step(np.arange(maximalD), xs_train[n], linestyle='-',alpha=0.5, color=colors[i], where="mid")

In [0]:
def train_discriminative_model(xs, ys, maxcond=1.e9):
  """
  train discriminative model y = B x + noise
  
  ## inputs:
  - xs - array of training data
  - ys - array of training labels
  - maxcond (optional) - limit on the condition number of the matrix inversions

  ## comments
  Informally speaking, this finds the B that minimizes || ys - B . xs || (plus
  some regularization), and returns the discriminative matrix B.

  ## bugs:
  - Properly, this should take in an estimate of the inverse variances. These
    might be some processing of the xivars and yivars; we need to figure that
    out.
  - Doesn't fit for a bias term (that is, a zero-offset).
  """
  return pseudo_solve(xs, ys, maxcond=maxcond).T

def train_trivial_generative_model(xs, ys, maxcond=1.e9):
  """
  say stuff here

  ## bugs:
  - Doesn't fit for a bias term (that is, a zero-offset).
  """
  return pseudo_solve(ys, xs, maxcond=maxcond)

def initialize_a_generative_model(Y, P):
  """
  HOGG say: This can be simplified with pseudo_inverse().
  """
  Z = np.linalg.lstsq(P, Y.T, rcond=None)[0]
  P1 = P.T
  proj_P = np.matmul(P1, np.linalg.solve(np.matmul(P1.T, P1), P1.T))
  M, humanK = P.shape
  proj_Pperp = np.identity(humanK) - proj_P
  return np.matmul(proj_P, Z), np.matmul(proj_Pperp, np.random.random(Z.shape))

def train_msv_generative_model(X, Y, P, iters=1000):
  """
  ## Inputs:
  X: N x D
  Y: N x M
  P: M x humanK

  ## Comments:
  among all solutions Z such that Y=Z*P.T (ie Y.T=P*Z.T)
  we minimize ||X - Z*A|| by alternating minimization
  we write Z = Z0 + Z1, proj_P*Z=Z0, proj_Pperp*Z = Z1 

  ## bugs:
  - Can be simplified with pseudo_inverse().
  - Doesn't use xivars, yivars.
  - Doesn't fit for a bias term (that is, a zero-offset).
  """
  proj_P= np.matmul(P.T, np.linalg.solve(np.matmul(P, P.T), P))
  M, humanK = P.shape
  proj_Pperp = np.identity(humanK) - proj_P
  
  Z0, Z1 = initialize_a_generative_model(Y, P)
  Z = Z0 + Z1
  last_cost=np.inf
  for i in range(iters):  
    #fix Z optimize for A
    A=np.linalg.lstsq(Z.T, X, rcond=None)[0]

    #fix A optimize for Z
    a = np.matmul(A.T, proj_Pperp.T)
    b = X.T- np.matmul(A.T, Z0)
    Z1=np.linalg.lstsq(a,b, rcond=None)[0]

    Z1=np.matmul(proj_Pperp, Z1)

    #convergence
    Z_old=Z
    Z=Z0+Z1
    #BUG: unstable! 
    #print('convergence, cost')
    cost= np.linalg.norm(X.T- np.matmul(A.T,Z)) 

    if cost >= last_cost:
      break
    last_cost=cost
    #print((np.linalg.norm(Z-Z_old), np.linalg.norm(X.T- np.matmul(A.T,Z)) + np.linalg.norm(Y.T -np.matmul(P,Z)) ))
  return  P @ np.linalg.solve(A @ A.T, A)

In [0]:
def z_step(X, Y, A, P, xivars, yivars):
  bigA = np.vstack((A, P))
  bigivars = np.append(xivars, yivars)
  bigX = np.hstack((X, Y))
  return pseudo_solve(bigA, bigX.T, weights=bigivars).T

def a_step(X, Z, xivars):
  """
  BUG: This doesn't currently use the ivars correctly.
  BUG: meaning: It treats the data as homoskedastic.
  """
  return pseudo_solve(Z, X).T

def dwh_cost(X, Y, Z, A, P, xivars, yivars):
  """
  ## Bugs:
  - Untested!
  """
  xresid = X - (A @ Z.T).T
  yresid = Y - (P @ Z.T).T
  return np.sum(xresid ** 2 * xivars) + np.sum(yresid ** 2 * yivars)

def train_dwh_generative_model(X, Y, P, xivars, yivars, maxiter=3, lltol=0.0001):
  """
  ## Bugs:
  - Initialization bad here. This should probably initialize at the internal
    state of the MSV generative model optimized earlier.
  - Takes forever to converge when K = M, apparently. This may be related to
    initialization also.
  - TOTAL HACK TO DEAL WITH infinite xivars.
  """
  Z0, Z1 = initialize_a_generative_model(Y, P)
  Z = (Z0 + Z1).T
  cost = np.inf
  unconverged = True
  A = 0.
  for ii in range(maxiter):
    old_cost = 1. * cost
    oldA = 1. * A
    A = a_step(X, Z, xivars)
    Z = z_step(X, Y, A, P, xivars, yivars)
    cost = dwh_cost(X, Y, Z, A, P, xivars, yivars)
    if cost > (old_cost + lltol):
      print("train_dwh_generative_model(): WARNING: cost went the wrong way!")
      print(ii, cost, old_cost)
      A = oldA # restore
    if cost > (old_cost - lltol):
      unconverged = False
      break
  if unconverged: print("train_dwh_generative_model(): WARNING: did not converge.")
  return P @ pseudo_inverse(A, weights=xivars, maxcond=1e9) # this maxcond is required for some reason

In [0]:
# run on the first trial case, for the fiducials
xs_train = maximalX[0, :fiducialN, :fiducialD]
ys_train = maximalY[0, :fiducialN, :]
xs_test = maximalXtest[:, :fiducialD]
ys_test = maximalYtest
xivars = maximalxivars[:fiducialD]
W = gods_estimator(P, maximalQ[:fiducialD, :], xivars)

print(xs_train.shape, ys_train.shape)
B = train_discriminative_model(xs_train, ys_train)
Hdagger = train_trivial_generative_model(xs_train, ys_train)
print(B.shape, Hdagger.shape)

# humanP = P
# humanP = np.hstack((P, np.zeros((M, 2))))
# humanP = P[:, :9]
humanK = K # stupid
assert humanK >= M
humanP = np.eye(humanK)[:M, :]
print(P.shape, humanP.shape)
G_msv = train_msv_generative_model(xs_train, ys_train, humanP, iters=1000)
G_dwh = train_dwh_generative_model(xs_train, ys_train, humanP, xivars, yivars, maxiter=1000)

ys_efficient_test = (W @ xs_test.T).T #W is information theory optimal
ys_msv_test = (G_msv @ xs_test.T).T
ys_dwh_test = (G_dwh @ xs_test.T).T
ys_discriminative_test = (B @ xs_test.T).T
ys_trivial_test = (Hdagger @ xs_test.T).T

In [0]:
def rms(x):
  return np.sqrt(np.mean(x * x))

m = 0
print("best possible", rms(ys_test[:, m] - ys_efficient_test[:, m]))
print("discriminative", rms(ys_test[:, m] - ys_discriminative_test[:, m]))
print("alt-trivial generative", rms(ys_test[:, m] - ys_trivial_test[:, m]))
print("MSV generative", rms(ys_test[:, m] - ys_msv_test[:, m]))
print("DWH generative", rms(ys_test[:, m] - ys_dwh_test[:, m]))

In [0]:
plt.figure(figsize=(20,4))
plt.subplot(151)
foo, bins, bar = plt.hist(ys_test[:, m] - ys_discriminative_test[:, m], bins=32)
plt.title("discriminative")
plt.subplot(155)
foo, bins, bar = plt.hist(ys_test[:, m] - ys_efficient_test[:, m], bins=bins)
plt.title("God's best")
plt.subplot(152)
plt.hist(ys_test[:, m] - ys_trivial_test[:, m], bins=bins)
plt.title("alt-trivial generative")
plt.subplot(153)
plt.hist(ys_test[:, m] - ys_msv_test[:, m], bins=bins)
plt.title("MSV generative")
plt.subplot(154)
plt.hist(ys_test[:, m] - ys_dwh_test[:, m], bins=bins)
plt.title("DWH generative")

In [0]:
def compute_mean_variance(ixs):
  xs = np.array(ixs)
  mean = np.mean(xs)
  var = np.mean(xs ** 2) - mean ** 2
  return mean, var

In [0]:
# make N plot at fiducialD

humanK = K # stupid
Ns = np.round((maximalN / 2 ** np.arange(0., 6.51, 0.25))[::-1]).astype(int)
dys_N = np.zeros((len(Ns), 4, Ntrial, Ntest, M))
for ii, N in enumerate(Ns):
  print("starting trials for N =", N)
  Bs, Gs_msv, Gs_dwh = [], [], []
  for trial in range(Ntrial):
    xs_train = maximalX[trial, :N, :fiducialD]
    ys_train = maximalY[trial, :N, :]
    xs_test = maximalXtest[:, :fiducialD]
    ys_test = maximalYtest
    xivars = maximalxivars[:fiducialD]
    W = gods_estimator(P, maximalQ[:fiducialD, :], xivars)
    Bs.append(train_discriminative_model(xs_train, ys_train))
    assert humanK >= M
    humanP = np.eye(humanK)[:M, :]
    Gs_msv.append(train_msv_generative_model(xs_train, ys_train, humanP, iters=1000))
    Gs_dwh.append(train_dwh_generative_model(xs_train, ys_train, humanP, xivars, yivars, maxiter=1000))
  dys_N[ii, 0] = np.array([ys_test - (B @ xs_test.T).T for B in Bs])
  dys_N[ii, 1] = np.array([ys_test - (G @ xs_test.T).T for G in Gs_msv])
  dys_N[ii, 2] = np.array([ys_test - (G @ xs_test.T).T for G in Gs_dwh])
  dys_N[ii, 3] = np.array([ys_test - (W @ xs_test.T).T for B in Bs]) # note stupid repeat of the same shit

In [0]:
mses_N = np.mean(np.mean(np.mean(dys_N ** 2, axis=-1), axis=-1), axis=-1)
plt.figure(figsize=(7, 5))
plt.subplot(111)
plt.plot(Ns, mses_N[:, 3], "k-")
plt.plot(Ns, mses_N[:, 0], "ko")
plt.plot(Ns, mses_N[:, 1], "ro")
plt.plot(Ns, mses_N[:, 2], "go")
plt.axvline(fiducialD, color="k", alpha=0.5)
plt.axvline(K, color="r", alpha=0.5)
plt.axvline(humanK, color="g", alpha=0.5)
plt.loglog()
plt.ylabel("MSE in prediction of TRUE y (variance + bias^2)")
plt.xlabel("number of training set objects N")
plt.title("varying N at fixed D")

In [0]:
# make D plot at fiducialN

humanK = K # stupid
Ds = np.round((maximalD / 2 ** np.arange(0., 6.51, 0.25))[::-1]).astype(int)
dys_D = np.zeros((len(Ds), 4, Ntrial, Ntest, M))
for ii, D in enumerate(Ds):
  print("starting trials for D =", D)
  Bs, Gs_msv, Gs_dwh = [], [], []
  for trial in range(Ntrial):
    xs_train = maximalX[trial, :fiducialN, :D]
    ys_train = maximalY[trial, :fiducialN, :]
    xs_test = maximalXtest[:, :D]
    ys_test = maximalYtest
    xivars = maximalxivars[:D]
    W = gods_estimator(P, maximalQ[:D, :], xivars)
    Bs.append(train_discriminative_model(xs_train, ys_train))
    assert humanK >= M
    humanP = np.eye(humanK)[:M, :]
    Gs_msv.append(train_msv_generative_model(xs_train, ys_train, humanP, iters=1000))
    Gs_dwh.append(train_dwh_generative_model(xs_train, ys_train, humanP, xivars, yivars, maxiter=1000))
  dys_D[ii, 0] = np.array([ys_test - (B @ xs_test.T).T for B in Bs])
  dys_D[ii, 1] = np.array([ys_test - (G @ xs_test.T).T for G in Gs_msv])
  dys_D[ii, 2] = np.array([ys_test - (G @ xs_test.T).T for G in Gs_dwh])
  dys_D[ii, 3] = np.array([ys_test - (W @ xs_test.T).T for B in Bs]) # note stupid repeat of the same shit

In [0]:
mses_D = np.mean(np.mean(np.mean(dys_D ** 2, axis=-1), axis=-1), axis=-1)
plt.figure(figsize=(7, 5))
plt.subplot(111)
plt.plot(Ds, mses_D[:, 3], "k-")
plt.plot(Ds, mses_D[:, 0], "ko")
plt.plot(Ds, mses_D[:, 1], "ro")
plt.plot(Ds, mses_D[:, 2], "go")
plt.axvline(fiducialN, color="k", alpha=0.5)
plt.axvline(K, color="r", alpha=0.5)
plt.axvline(humanK, color="g", alpha=0.5)
plt.loglog()
plt.ylabel("MSE in prediction of TRUE y (variance + bias^2)")
plt.xlabel("number of image pixels D")
plt.title("varying D at fixed N")

# OLD STUFF

# OLD STUFF FOLLOWS

In [0]:
# make things to histogram
Bs, Gs_msv, Gs_dwh = trainings # unpack
Ntrials, M, D = Bs.shape
Ntest = 2 ** 5 + 5
dys_discriminative = np.zeros((Ntrials, Ntest, M))
dys_msv = np.zeros((Ntrials, Ntest, M))
dys_dwh = np.zeros((Ntrials, Ntest, M))
for nn in range(Ntrials):
  xtest, ytest = make_data_set(Ntest, P, Q, xivars, yivars) # generate with God's knowledge
  dys_discriminative[nn] = ytest - (Bs[nn] @ xtest.T).T
  dys_msv[nn] = ytest - (Gs_msv[nn] @ xtest.T).T
  dys_dwh[nn] = ytest - (Gs_dwh[nn] @ xtest.T).T

In [0]:
plt.figure(figsize=(15,5))
plt.subplot(131)
foo, bins, bar = plt.hist(dys_discriminative.reshape((Ntrials, Ntest)).T, bins=Ntest // 5, stacked=True)
plt.title("discriminative prediction of y")
plt.subplot(132)
plt.hist(dys_msv.reshape((Ntrials, Ntest)).T, bins=bins, stacked=True)
plt.title("MSV generative prediction of y")
plt.subplot(133)
plt.hist(dys_dwh.reshape((Ntrials, Ntest)).T, bins=bins, stacked=True)
plt.title("DWH generative prediction of y")

In [0]:
# compute means and variances
means = np.mean(trainings, axis=1)
mean2s = np.mean(trainings ** 2, axis=1)
variances = mean2s - means ** 2
variances = np.sum(np.sum(variances, axis=-1), axis=-1)
print(variances)

In [0]:
# OLD plotting macro
def plot_inference(labels, inference, title, name):
  foo = np.array([-20, 20])
  bar = 1.1 * np.max(np.abs(labels))
  plt.figure(figsize=(15,5))
  plt.subplot(131)
  plt.scatter(labels, inference)
  plt.title(title + " inference (vs labels) for " + name)
  plt.xlabel("label slope")
  plt.ylabel("inferred slope")
  plt.plot(foo, foo, "k-", alpha=0.25)
  plt.xlim(-bar, bar)
  plt.ylim(-bar, bar)
  print("RMSE vs labels:", rms(labels - inference))
  #plt.subplot(132)
  #plt.scatter(truth, inference)
  #plt.title(title + " inference (vs truth) for " + name)
  #plt.xlabel("true slope")
  #plt.ylabel("inferred slope")
  #plt.plot(foo, foo, "k-", alpha=0.25)
  #plt.xlim(-bar, bar)
  #plt.ylim(-bar, bar)
  #print("RMSE vs truth:", rms(truth - inference))
  plt.subplot(132)
  plt.scatter(labels, inference - labels)
  plt.title(title + " inference for " + name)
  plt.xlabel("label slope")
  plt.ylabel("residual (inferred - label)")
  plt.plot(foo, 0. * foo, "k-", alpha=0.25)
  plt.xlim(-bar, bar)
  plt.ylim(-0.3 * bar, 0.3 * bar)
  plt.tight_layout()

In [0]:
# plot and report outcomes for the efficient estimator
# this should be the best you can possibly do
m = 0
plot_inference(ys_test[:, m], ys_efficient_test[:, m],
               "", "the efficient estimator")

In [0]:
# plot and report outcomes for the discriminative model
plot_inference(ys_test[:, m], ys_discriminative_test[:, m],
               "", "the discriminative model")

In [0]:
plot_inference(ys_test[:, m], ys_msv_test[:, m],
               "", "the MSV generative model")

In [0]:
plot_inference(ys_test[:, m], ys_trivial_test[:, m],
               "", "the alt-trivial generative model")

In [0]:
G_dwh = train_dwh_generative_model(xs_train, ys_train, humanP, xivars, yivars)

In [0]:
ys_dwh_test = (G_dwh @ xs_test.T).T
plot_inference(ys_test[:, m], ys_dwh_test[:, m],
               "", "the DWH generative model")

In [0]:
fig, ax = plt.subplots(1,4,sharey=True, figsize=(18,4.5))
for m in range(M):
  ax[0].plot(W[m, :], alpha=0.75)
ax[0].set_title("true (information-theory-optimal) result")
for m in range(M):
  ax[1].plot(B[m, :], alpha=0.75)
ax[1].set_title("discriminative result")
for m in range(M):
  ax[2].plot(G_msv[m, :], alpha=0.75)
ax[2].set_title("MSV generative result")
for m in range(M):
  ax[3].plot(Hdagger[m, :], alpha=0.75)
ax[3].set_title("alt-trivial generative result")
ax[3].set_ylim(np.min(W), np.max(W))

In [0]:
fig, ax = plt.subplots(1,4,sharey=True, figsize=(18,4.5))
for m in range(M):
  ax[0].plot(W[m, :], alpha=0.75)
ax[0].set_title("true (information-theory-optimal) result")
for m in range(M):
  ax[1].plot(G_msv[m, :], alpha=0.75)
ax[1].set_title("MSV generative result")
for m in range(M):
  ax[2].plot(G_dwh[m, :], alpha=0.75)
ax[2].set_title("DWH generative result")
for m in range(M):
  ax[3].plot(Hdagger[m, :], alpha=0.75)
ax[3].set_title("alt-trivial generative result")
ax[3].set_ylim(np.min(W), np.max(W))

In [0]:
def train_many_times(Ntrain, humanP, P, Q, xivars, yivars, Ntrials=2**5):
  """
  Perform many trainings of each model for the purposes of empirically testing
  biases and variances.

  ## inputs:
  - Ntrain - training set size N
  - humanP - human-set projector P
  - P - God's projector P
  - Q - God's embeddor Q
  - xivars - diagonal of C_x\inv
  - yivars - diagonal of C_y\inv
  - Ntrials (optional) - number of independent trials
  """
  Bs, Gs_msv, Gs_dwh = [], [], []
  for trial in range(Ntrials):
    xtrain, ytrain = make_data_set(Ntrain, P, Q, xivars, yivars) # generate with God's knowledge
    Bs.append(train_discriminative_model(xtrain, ytrain))
    Gs_msv.append(train_msv_generative_model(xtrain, ytrain, humanP, iters=1000)) # infer with human choices
    Gs_dwh.append(train_dwh_generative_model(xtrain, ytrain, humanP, xivars, yivars, maxiter=1000))
  return np.array([np.array(Bs), np.array(Gs_msv), np.array(Gs_dwh)])